# Download query from Dune

#### Maria Silva, April 2024

Note that we are running this from google colab!!! This means that you will not be able to run this notebook without access to the secrets and a google colab enviroment.

## Imports and secrets

In [ ]:
import json
import pickle
import requests
import pandas as pd
from google.colab import userdata

In [ ]:
DUNE_API_KEY = userdata.get('X-DUNE-API-KEY')
SECRETE_GCP_KEY = userdata.get('secret_gcp_key')

with open('secret_gcp_key.json', 'w') as f:
    f.write(SECRETE_GCP_KEY)

## Install gfuse and mount GCP bucket

In [ ]:
%%capture
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [ ]:
!mkdir -p research-analysis-422212-storage
!gcsfuse --key-file secret_gcp_key.json  research-analysis-422212-storage research-analysis-422212-storage
!touch research-analysis-422212-storage/test

## Download data

In [ ]:
# Query settings
query_id = "3685205"
out_file_name = "zksync_gas_data.csv"

# Request
url = f"https://api.dune.com/api/v1/query/{query_id}/results?allow_partial_results=true&limit=250000"
headers = {"X-DUNE-API-KEY": DUNE_API_KEY}

# Pagination loop
data_df = pd.DataFrame()
n = 0
while url:
  response = requests.request("GET", url, headers=headers)
  results = json.loads(response.text)
  if "error" in results:
    if results["error"] == 'internal error':
      response = requests.request("GET", url, headers=headers)
      results = json.loads(response.text)
  temp_data = results['result']['rows']
  temp_data_df = pd.DataFrame(temp_data)
  data_df = pd.concat([data_df, temp_data_df], ignore_index=True)
  if n % 100 == 0:
    data_df.to_csv(f"./research-analysis-422212-storage/temp_{out_file_name}", index=False)
    print(f"Checkpointed at {url}")
  n += 1
  if 'next_uri' in results:
    url = results['next_uri']
  else:
    url = None

data_df.to_csv(f"./research-analysis-422212-storage/{out_file_name}", index=False)
print("Done")

In [ ]:
data_df.info()

In [ ]:
data_df.head()

In [ ]:
data_df.tail()